In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
import os
from langchain_nomic import NomicEmbeddings
from dotenv import load_dotenv
# from openai.error import RateLimitError
from langchain_openai import ChatOpenAI
import networkx as nx
import fitz
from typing import List
import numpy as np
from knowledge_graph import KnowledgeGraph
from query_engine import QueryEngine
from langchain.document_loaders import  PyPDFLoader

load_dotenv()  # Load environment variables from .env file

api_key = os.getenv("NOMIC_API_KEY")

path = "/home/name-1/AI-Agent/RAG_Project/RAG_Project/data/Understanding_Climate_Change.pdf"
loader = PyPDFLoader(path)
documents = loader.load()
documents = documents[:10]


# DocumentProcessor's process_documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
embeddings =  NomicEmbeddings(model="nomic-embed-text-v1.5",)
splits = text_splitter.split_documents(documents)
vector_store = FAISS.from_documents(splits, embeddings)

/home/name-1/AI-Agent/RAG_Project/venv/lib/python3.11/site-packages/deepeval/__init__.py:49: UserWarning: You are using deepeval version 1.1.4, however version 1.1.6 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [2]:
knowledge_graph = KnowledgeGraph()
llm = ChatOpenAI(
                model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
                base_url="http://10.2.125.37:1234/v1",
                api_key="lm-studio"
            )

In [3]:
import networkx as nx
# _add_nodes(splits)

graph = nx.Graph()

for i, split in enumerate(splits):
    graph.add_node(i, content=split.page_content)

In [5]:
graph

In [7]:
# embeddings_xx = _create_embeddings(splits, embeddings)

texts = [split.page_content for split in splits]
embeddings_xx = embeddings.embed_documents(texts)
# embeddings_xx

In [ ]:
    # def build_graph(self, splits, llm, embedding_model):
    #     """
    #     Builds the knowledge graph by adding nodes, creating embeddings, extracting concepts, and adding edges.
        
    #     Args:
    #     - splits (list): A list of document splits.
    #     - llm: An instance of a large language model.
    #     - embedding_model: An instance of an embedding model.
        
    #     Returns:
    #     - None
    #     """
    #     self._add_nodes(splits)
    #     embeddings = self._create_embeddings(splits, embedding_model)
    #     self._extract_concepts(splits, llm)
    #     self._add_edges(embeddings)

In [8]:
# _extract_concepts(splits, llm)
from concurrent.futures import ThreadPoolExecutor, as_completed
import tqdm as tqdm
from langchain_core.prompts import PromptTemplate
# from typing import 
from langchain_core.pydantic_v1 import BaseModel, Field
class Concepts(BaseModel):
    concepts_list: List[str] = Field(description="List of concepts")




def _extract_concepts_and_entities(self, content, llm):
        """
        Extracts concepts and named entities from the content using spaCy and a large language model.
        
        Args:
        - content (str): The content from which to extract concepts and entities.
        - llm: An instance of a large language model.
        
        Returns:
        - list: A list of extracted concepts and entities.
        """
        if content in self.concept_cache:
            return self.concept_cache[content]
        
        # Extract named entities using spaCy
        doc = self.nlp(content)
        named_entities = [ent.text for ent in doc.ents if ent.label_ in ["PERSON", "ORG", "GPE", "WORK_OF_ART"]]
        
        # Extract general concepts using LLM
        concept_extraction_prompt = PromptTemplate(
            input_variables=["text"],
            template="Extract key concepts (excluding named entities) from the following text:\n\n{text}\n\nKey concepts:"
        )
        concept_chain = concept_extraction_prompt | llm.with_structured_output(Concepts)
        general_concepts = concept_chain.invoke({"text": content}).concepts_list
        
        # Combine named entities and general concepts
        all_concepts = list(set(named_entities + general_concepts))
        
        self.concept_cache[content] = all_concepts
        return all_concepts



# with ThreadPoolExecutor() as executor:
#     future_to_node = {executor.submit(_extract_concepts_and_entities, split.page_content, llm): i 
#                         for i, split in enumerate(splits)}
    
#     for future in tqdm(as_completed(future_to_node), total=len(splits), desc="Extracting concepts and entities"):
#         node = future_to_node[future]
#         concepts = future.result()
#         graph.nodes[node]['concepts'] = concepts


In [9]:
import spacy
from spacy.cli import download
def _load_spacy_model():
    """
    Loads the spaCy NLP model, downloading it if necessary.
    
    Args:
    - None
    
    Returns:
    - spacy.Language: An instance of a spaCy NLP model.
    """
    try:
        return spacy.load("en_core_web_sm")
    except OSError:
        print("Downloading spaCy model...")
        download("en_core_web_sm")
        return spacy.load("en_core_web_sm")

In [11]:
nlp = _load_spacy_model()

In [12]:
# _extract_concepts_and_entities(split.page_content, llm)

concept_cache = {}
content = split.page_content

if content in concept_cache:
    concept_cache[content]


doc = nlp(content)
# doc
named_entities = [ent.text for ent in doc.ents if ent.label_ in ["PERSON", "ORG", "GPE", "WORK_OF_ART"]]
named_entities
# Extract general concepts using LLM
concept_extraction_prompt = PromptTemplate(
    input_variables=["text"],
    template="Extract key concepts (excluding named entities) from the following text:\n\n{text}\n\nKey concepts:"
)
concept_extraction_prompt
concept_chain = concept_extraction_prompt | llm.with_structured_output(Concepts)
concept_chain

PromptTemplate(input_variables=['text'], template='Extract key concepts (excluding named entities) from the following text:\n\n{text}\n\nKey concepts:')
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7c4dd422e090>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7c4ddc110a50>, root_client=<openai.OpenAI object at 0x7c4dd6145cd0>, root_async_client=<openai.AsyncOpenAI object at 0x7c4ddc113790>, model_name='lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF', openai_api_key=SecretStr('**********'), openai_api_base='http://10.2.125.37:1234/v1', openai_proxy=''), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Concepts', 'description': '', 'parameters': {'type': 'object', 'properties': {'concepts_list': {'description': 'List of concepts', 'type': 'array', 'items': {'type': 'string'}}}, 'required': ['concepts_list']}}}], 'parallel_tool_calls': False, 'tool_choice': {'type': 'function', 'function'

In [14]:
content

'caps, and pollution controls. Enforcement and compliance are critical for their  effectiveness.  \nLocal and Community Initiatives  \nUrban Climate Action  \nCities play a pivotal role in climate action due to their high population densities and \neconomic activities. Urban climate initiatives include sustainable transportation systems, \ngreen building standards, and climate -resilient infrastructure. Community eng agement and \nparticipatory planning are essential for successful implementation.  \nCommunity -Based Conservation'

In [17]:
general_concepts = concept_chain.invoke({"text": content})
# general_concepts = general_concepts.concepts_list
general_concepts

In [18]:
print(general_concepts)

None


In [ ]:
_add_edges(embeddings_xx)

In [4]:
knowledge_graph.build_graph(splits, llm, embeddings)

Extracting concepts and entities:   0%|          | 0/30 [00:11<?, ?it/s]


In [ ]:
query_engine = QueryEngine(vector_store, knowledge_graph, llm)